# Simple Workflow with pyiron_base

## Define workflow with pyiron_base

In [1]:
from pyiron_base import Project, job

In [2]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [3]:
from simple_workflow import (
    add_x_and_y as _add_x_and_y, 
    add_x_and_y_and_z as _add_x_and_y_and_z,
)

In [4]:
workflow_json_filename = "pyiron_base_simple.json"

In [5]:
add_x_and_y = job(_add_x_and_y, output_key_lst=["x", "y", "z"])
add_x_and_y_and_z = job(_add_x_and_y_and_z)

In [6]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
obj = add_x_and_y(x=1, y=2, pyiron_project=pr)

In [8]:
w = add_x_and_y_and_z(x=obj.output.x, y=obj.output.y, z=obj.output.z, pyiron_project=pr)

In [9]:
write_workflow_json(delayed_object=w, file_name=workflow_json_filename)

In [10]:
!cat {workflow_json_filename}

{"nodes": {"0": "simple_workflow.add_x_and_y_and_z", "1": "simple_workflow.add_x_and_y", "2": 1, "3": 2}, "edges": [{"target": 0, "targetHandle": "x", "source": 1, "sourceHandle": "x"}, {"target": 1, "targetHandle": "x", "source": 2, "sourceHandle": null}, {"target": 1, "targetHandle": "y", "source": 3, "sourceHandle": null}, {"target": 0, "targetHandle": "y", "source": 1, "sourceHandle": "y"}, {"target": 0, "targetHandle": "z", "source": 1, "sourceHandle": "z"}]}

## Load Workflow with aiida

In [11]:
from aiida import load_profile

load_profile()

Profile<uuid='c08748df7ab74ec08c14e5f16cebc987' name='pwd'>

In [12]:
from python_workflow_definition.aiida import load_workflow_json

In [13]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [14]:

wg.run()

03/22/2025 05:14:45 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|continue_workgraph]: tasks ready to run: pickle_node3,pickle_node4
03/22/2025 05:14:47 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 60, 67
03/22/2025 05:14:58 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|update_task_state]: Task: pickle_node3, type: PYTHONJOB, finished.
03/22/2025 05:14:58 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|update_task_state]: Task: pickle_node4, type: PYTHONJOB, finished.


invalid state


03/22/2025 05:14:59 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|continue_workgraph]: tasks ready to run: add_x_and_y2
03/22/2025 05:14:59 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 82
03/22/2025 05:15:06 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|update_task_state]: Task: add_x_and_y2, type: PYTHONJOB, finished.
03/22/2025 05:15:07 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|continue_workgraph]: tasks ready to run: add_x_and_y_and_z1
03/22/2025 05:15:07 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 95
03/22/2025 05:15:15 PM <24142> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [53|WorkGraphEn

## Load Workflow with jobflow

In [15]:
from python_workflow_definition.jobflow import load_workflow_json

/home/jan/mambaforge/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/jan/mambaforge/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [16]:
from jobflow.managers.local import run_locally

In [17]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [18]:
result = run_locally(flow)
result

2025-03-22 17:15:17,020 INFO Started executing jobs locally
2025-03-22 17:15:17,608 INFO Starting job - add_x_and_y (0ff79865-353c-4e5c-9e0c-6d9ed576e001)
2025-03-22 17:15:17,610 INFO Finished job - add_x_and_y (0ff79865-353c-4e5c-9e0c-6d9ed576e001)
2025-03-22 17:15:17,612 INFO Starting job - add_x_and_y_and_z (1d6e359b-898d-4784-a86d-ca858fc2fcfd)
2025-03-22 17:15:17,615 INFO Finished job - add_x_and_y_and_z (1d6e359b-898d-4784-a86d-ca858fc2fcfd)
2025-03-22 17:15:17,615 INFO Finished executing jobs locally


{'0ff79865-353c-4e5c-9e0c-6d9ed576e001': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jan/notebooks/2025/2025-03-22-compare-workflows'))},
 '1d6e359b-898d-4784-a86d-ca858fc2fcfd': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jan/notebooks/2025/2025-03-22-compare-workflows'))}}